In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
plt.rcParams['figure.dpi'] = 200

In [2]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [3]:
x_list = []
y_list = []

# The bigger the time step, the more the history. 
TIME_STEPS = 16
SAMPLE_STEPS = 4

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]

# NOTE: Uncomment if using polynomial
poly_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

# NOTE: Uncomment if using markers
marker_labels = ["M1X", "M1Y", "M2X", "M2Y", "M3X", "M3Y",
            "M4X", "M4Y", "M5X", "M5Y", "M6X", "M6Y",
            "M7X", "M7Y", "M8X", "M8Y", "M9X", "M9Y",
            "M10X", "M10Y"]

y_labels = marker_labels

INPUT_DIM = len(x_labels)
OUTPUT_DIM = len(y_labels)

def get_index_groups(df):
    return df.groupby(df.index.to_series().diff().ne(1).cumsum()).groups

def expand_time_steps(data, time_steps, sample_steps):
    entries = data.shape[0] - time_steps + 1
    rows = int(time_steps / sample_steps)
    cols = data.shape[1]
    new_arr = np.zeros((entries, rows, cols))
    for i in range(data.shape[0] - time_steps + 1):
        full_arr = data.iloc[i:i + time_steps].to_numpy()
        new_arr[i, :, :] = full_arr[sample_steps - 1::sample_steps]
    return new_arr

def normalize_2d(data):
    min_vals = []
    max_vals = []
    rows = data.shape[0]
    cols = data.shape[1]
    for c in range(cols): 
        mx = data[0][c]
        mn = mx
        for r in range(rows):
            val = data[r][c]
            if val > mx:
                mx = val
            if val < mn:
                mn = val
        min_vals.append(mn)
        max_vals.append(mx)
        for r in range(rows):
            val = data[r][c]
            data[r][c] = (val - mn) / (mx - mn)
    return min_vals, max_vals
    
def normalize_3d(data):
    min_vals = []
    max_vals = []
    entries = data.shape[0]
    rows = data.shape[1]
    cols = data.shape[2]
    for c in range(cols):
        mx = data[0][0][c]
        mn = mx
        for e in range(entries):
            for r in range(rows):
                val = data[e][r][c]
                if val > mx:
                    mx = val
                if val < mn:
                    mn = val
        min_vals.append(mn)
        max_vals.append(mx)
        for e in range(entries):
            for r in range(rows):
                val = data[e][r][c]
                data[e][r][c] = (val - mn) / (mx - mn)
    return min_vals, max_vals

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    # NOTE: Handle when the robot goes out of frame.
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices])
    marker_df = marker_df.drop(marker_df.index[zero_indices])
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_groups = get_index_groups(data_df)
    marker_groups = get_index_groups(marker_df)
    poly_groups = get_index_groups(poly_df)
    
    for dg in data_groups:
        idx_list = data_groups[dg]
        sub_data_df = data_df.loc[idx_list[0]:idx_list[-1]+1, x_labels]
        sub_marker_df = marker_df.loc[idx_list[0]:idx_list[-1]+1, marker_labels]
        sub_poly_df = poly_df.loc[idx_list[0]:idx_list[-1]+1, poly_labels]
        
        sub_data_df_exp = expand_time_steps(sub_data_df, TIME_STEPS, SAMPLE_STEPS)
        sub_marker_df_exp = sub_marker_df[TIME_STEPS-1:]
        sub_poly_df_exp = sub_poly_df[TIME_STEPS-1:]
        
        print(sub_data_df_exp.shape)
        print(sub_marker_df_exp.shape)
        print(sub_poly_df_exp.shape)
        
        x_list.append(sub_data_df_exp)
        
        if y_labels == poly_labels:
            y_list.append(sub_poly_df_exp.to_numpy())
        elif y_labels == marker_labels:
            y_list.append(sub_marker_df_exp.to_numpy())
        
x_data = np.concatenate(x_list, axis=0)
y_data = np.concatenate(y_list, axis=0)

x_mins, x_maxes = normalize_3d(x_data)
y_mins, y_maxes = normalize_2d(y_data)

x_data_shuffled, y_data_shuffled = sklearn.utils.shuffle(x_data, y_data)

data_length = len(x_data_shuffled)
split_percent = 0.8
train_test_split = int(data_length * split_percent)

x_train = x_data_shuffled[:train_test_split]
y_train = y_data_shuffled[:train_test_split]

x_test = x_data_shuffled[train_test_split:]
y_test = y_data_shuffled[train_test_split:]

INPUT_TIME_STEPS = x_train.shape[1]


(466, 4, 4)
(466, 20)
(466, 7)
(491, 4, 4)
(491, 20)
(491, 7)
(496, 4, 4)
(496, 20)
(496, 7)
(603, 4, 4)
(603, 20)
(603, 7)
(585, 4, 4)
(585, 20)
(585, 7)
(511, 4, 4)
(511, 20)
(511, 7)
(425, 4, 4)
(425, 20)
(425, 7)
(442, 4, 4)
(442, 20)
(442, 7)
(269, 4, 4)
(269, 20)
(269, 7)
(693, 4, 4)
(693, 20)
(693, 7)
(234, 4, 4)
(234, 20)
(234, 7)
(724, 4, 4)
(724, 20)
(724, 7)


In [4]:
# Have to take TIME_STEPS off the top of *every* run. Not just them all together.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4751, 4, 4)
(4751, 20)
(1188, 4, 4)
(1188, 20)


In [5]:
model = Sequential()
#model.add(LSTM(64, input_shape=(INPUT_TIME_STEPS, INPUT_DIM), activation='relu', return_sequences=True))
model.add(LSTM(64, input_shape=(INPUT_TIME_STEPS, INPUT_DIM), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(OUTPUT_DIM, activation='relu'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                17664     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                5

In [6]:
BATCH_SIZE = 8
EPOCHS = 100
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0923 - accuracy: 0.1022
Epoch 2/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0219 - accuracy: 0.2402
Epoch 3/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0160 - accuracy: 0.3487
Epoch 4/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0152 - accuracy: 0.3935
Epoch 5/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0136 - accuracy: 0.4006
Epoch 6/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0125 - accuracy: 0.4404
Epoch 7/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0118 - accuracy: 0.4524
Epoch 8/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0108 - accuracy: 0.4706
Epoch 9/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0106 - accuracy: 0.4944
Epoch 10/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0107 - accura

594/594 [==============================] - 1s 1ms/step - loss: 0.0060 - accuracy: 0.6028
Epoch 82/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0059 - accuracy: 0.5767
Epoch 83/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0058 - accuracy: 0.5885
Epoch 84/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0056 - accuracy: 0.5903
Epoch 85/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0077 - accuracy: 0.5880
Epoch 86/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0068 - accuracy: 0.5984
Epoch 87/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0054 - accuracy: 0.5846
Epoch 88/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0055 - accuracy: 0.5798
Epoch 89/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0060 - accuracy: 0.5969
Epoch 90/100
594/594 [==============================] - 1s 1ms/step - loss: 0.0056 - accuracy: 

In [7]:
loss, accuracy = model.evaluate(x_test, y_test)

38/38 [==============================] - 0s 627us/step - loss: 0.0023 - accuracy: 0.6473


In [18]:
pickle.dump(history.history["loss"], open("objects/rnn_loss.p", "wb"))

In [8]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [35]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    x_select = np.expand_dims(x_test[idx], axis=0)
    #x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = y_test[idx]
    y_select = rescale_output(y_select)
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys, label="actual")
    plt.plot(xp, yp, label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Shape vs. Actual Shape")
    plt.show()
    

interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1187, 1))>

In [9]:
num_tests = len(x_test)
def evaluate_markers(idx=(0,num_tests-1,1)):
    x_select = np.expand_dims(x_test[idx], axis=0)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    
    y_select = y_test[idx]
    y_select = rescale_output(y_select)
    
    num_markers = int(len(y_pred) / 2)
    
    x_coord_select = [0]
    y_coord_select = [0]
    x_coord_pred = [0]
    y_coord_pred = [0]
    
    for i in range(num_markers):
        x_idx = i * 2
        y_idx = (i * 2) + 1
        x_coord_select.append(y_select[x_idx])
        y_coord_select.append(y_select[y_idx])
        x_coord_pred.append(y_pred[x_idx])
        y_coord_pred.append(y_pred[y_idx])
    
    plt.plot(x_coord_select, y_coord_select, "-o", label="actual")
    plt.plot(x_coord_pred, y_coord_pred, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
        
        
interact(evaluate_markers)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_markers(idx=(0, 1187, 1))>

In [145]:
a = np.array([1,2,3,4,5,6,7,8])
a[3::4]

array([4, 8])

In [173]:
model.save("./models/first_rnn")

INFO:tensorflow:Assets written to: ./models/first_rnn/assets
